In [3]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [4]:
%pip install -qU datasets transformers torch aiohttp numpy tqdm pytest datasets torch transformers evaluate

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu116 requires torch==1.12.1, but you have torch 2.1.2 which is incompatible.
thinc 8.1.7 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.5.3 which is incompatible.
spacy 3.4.1 requires pydantic!=1.8,!=1.8.1,<1.10.0,>=1.7.4, but you have pydantic 2.5.3 which is incompatible.
spacy 3.4.1 requires typer<0.5.0,>=0.3.0, but you have typer 0.9.0 which is incompatible.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!apt update
!apt install -y mono-devel
!ln -s /usr/bin/mono-csc /usr/bin/csc

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://deb.nodesource.com/node_16.x focal InRelease                     
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]      
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease                         
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease33m
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64 Packages [32.5 kB]
Get:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3770 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [3294 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1461 kB]
Fetched 8786 kB in 2s (5090 kB/s)                          
Reading package lists... Done
Building dependency

In [6]:
!csc --version

Mono C# compiler version 6.8.0.105


In [7]:
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate

/usr/local/lib/python3.9/dist-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [8]:
LANG = "cs"
MODEL_NAME = "stabilityai/stable-code-3b"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, trust_remote_code=True, torch_dtype="auto")
model.to("cuda")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

StableLMEpochForCausalLM(
  (model): StableLMEpochModel(
    (embed_tokens): Embedding(50304, 2560)
    (layers): ModuleList(
      (0-31): 32 x DecoderLayer(
        (self_attn): Attention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (rotary_emb): RotaryEmbedding()
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_a

In [10]:
problems = datasets.load_dataset("nuprl/MultiPL-E", f"humaneval-{LANG}", trust_remote_code=True)

Run from here down to try a different Multi PL-E problem

In [15]:
problem_len = len(problems['test'])
mid_tok = tokenizer("<fim_middle>")['input_ids'][0]
references = []
predictions = []

for x in range(problem_len):
  problem = problems['test'][x]
  prompt = problem['prompt']
  tests = problem['tests']
  fim = f"<fim_prefix>{prompt}<fim_suffix>{tests}<fim_middle>"
  inputs = tokenizer(fim, return_tensors="pt").to(model.device)
  tokens = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.2,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id
  )
  mid_pos = (tokens[0]==mid_tok).nonzero().item()
  masked_index = torch.nonzero(tokens[0] == mid_tok, as_tuple=False)[0].item()
  fim = tokenizer.decode(tokens[0][masked_index:], skip_special_tokens=True)
  with open("program.cs", "w", encoding='utf8') as doc:
    doc.write(prompt)
    doc.write(fim)
    doc.write(tests)
  import subprocess
  build = subprocess.run(['csc','/d:DEBUG','-r:System.Numerics.dll', 'program.cs', '/out:program.exe'], capture_output=True)
  references.append(1 if build.returncode == 0 else 0)
  predictions.append(1)
  print(problem['name']+f'({x}): {"✔️" if build.returncode == 0 else "❌"}')

HumanEval_23_strlen(0): ✔️
HumanEval_89_encrypt(1): ✔️
HumanEval_95_check_dict_case(2): ✔️
HumanEval_85_add(3): ✔️
HumanEval_140_fix_spaces(4): ✔️
HumanEval_63_fibfib(5): ✔️
HumanEval_151_double_the_difference(6): ❌
HumanEval_22_filter_integers(7): ✔️
HumanEval_41_car_race_collision(8): ✔️
HumanEval_17_parse_music(9): ✔️
HumanEval_79_decimal_to_binary(10): ❌
HumanEval_14_all_prefixes(11): ❌
HumanEval_53_add(12): ✔️
HumanEval_159_eat(13): ✔️
HumanEval_115_max_fill(14): ❌
HumanEval_160_do_algebra(15): ✔️
HumanEval_27_flip_case(16): ❌
HumanEval_105_by_length(17): ✔️
HumanEval_25_factorize(18): ✔️
HumanEval_96_count_up_to(19): ❌
HumanEval_34_unique(20): ✔️
HumanEval_74_total_match(21): ✔️
HumanEval_35_max_element(22): ✔️
HumanEval_132_is_nested(23): ❌
HumanEval_113_odd_count(24): ❌
HumanEval_109_move_one_ball(25): ✔️
HumanEval_107_even_odd_palindrome(26): ❌
HumanEval_138_is_equal_to_sum_even(27): ✔️
HumanEval_62_derivative(28): ✔️
HumanEval_126_is_sorted(29): ✔️
HumanEval_161_solve(30): ✔️

In [16]:
accuracy = evaluate.load("accuracy")
results = accuracy.compute(references=references, predictions=predictions)
print(results)

{'accuracy': 0.7531645569620253}
